In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.metrics import classification_report

In [38]:
test_data = pd.read_csv('../data/titanic/test.csv')
train_data = pd.read_csv('../data/titanic/train.csv')


In [39]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [40]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [41]:
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace=True)
train_data['Age'].fillna(train_data['Age'].median(), inplace=True)

test_data['Embarked'].fillna(test_data['Embarked'].mode()[0], inplace=True)
test_data['Age'].fillna(test_data['Age'].median(), inplace=True)

In [42]:
# standardize the age and fare
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_data['Age'] = scaler.fit_transform(train_data[['Age']])
train_data['Fare'] = scaler.fit_transform(train_data[['Fare']])
test_data['Age'] = scaler.fit_transform(test_data[['Age']])
test_data['Fare'] = scaler.fit_transform(test_data[['Fare']])
train_data.head() 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,-0.565736,1,0,A/5 21171,-0.502445,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,0.663861,1,0,PC 17599,0.786845,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,-0.258337,0,0,STON/O2. 3101282,-0.488854,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,0.433312,1,0,113803,0.420730,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,0.433312,0,0,373450,-0.486337,NaN,S


In [43]:
from sklearn import preprocessing
la_embarked = preprocessing.LabelEncoder()
la_embarked.fit(['S','C','Q'])
train_data['Embarked'] = la_embarked.transform(train_data['Embarked'])
test_data['Embarked'] = la_embarked.transform(test_data['Embarked'])

la_sex = preprocessing.LabelEncoder()
la_sex.fit(['male','female'])
train_data['Sex'] = la_sex.transform(train_data['Sex'])
test_data['Sex'] = la_sex.transform(test_data['Sex'])

train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,-0.565736,1,0,A/5 21171,-0.502445,NaN,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,0.663861,1,0,PC 17599,0.786845,C85,0
2,3,1,3,"Heikkinen, Miss. Laina",0,-0.258337,0,0,STON/O2. 3101282,-0.488854,NaN,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,0.433312,1,0,113803,0.420730,C123,2
4,5,0,3,"Allen, Mr. William Henry",1,0.433312,0,0,373450,-0.486337,NaN,2


In [48]:
y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch", 'Fare', 'Embarked', 'Age']
X = train_data[features]
X_test = test_data[features]

X[:5]

,Pclass,Sex,SibSp,Parch,Fare,Embarked,Age
0,3,1,1,0,-0.502445,2,-0.565736
1,1,0,1,0,0.786845,0,0.663861
2,3,0,0,0,-0.488854,2,-0.258337
3,1,0,1,0,0.420730,2,0.433312
4,3,1,0,0,-0.486337,2,0.433312


In [54]:
X.fillna(0)

,Pclass,Sex,SibSp,Parch,Fare,Embarked,Age
0,3,1,1,0,-0.502445,2,-0.565736
1,1,0,1,0,0.786845,0,0.663861
2,3,0,0,0,-0.488854,2,-0.258337
3,1,0,1,0,0.420730,2,0.433312
4,3,1,0,0,-0.486337,2,0.433312
...,...,...,...,...,...,...,...
886,2,1,0,0,-0.386671,2,-0.181487
887,1,0,0,0,-0.044381,2,-0.796286
888,3,0,1,2,-0.176263,2,-0.104637
889,1,1,0,0,-0.044381,0,-0.258337


In [56]:
from sklearn.ensemble import HistGradientBoostingClassifier

model = HistGradientBoostingClassifier().fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submissions/titanic/submissionHGBC.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
